In [31]:
import sklearn as sk
import numpy as np
import pandas as pd
import rasterio, os, napari
from sklearn import linear_model as lm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans

In [2]:
with open(r"C:\Users\marke\Downloads\data (1).npy", "rb") as file:
    data = np.load(file)

In [3]:
with open(r"C:\Users\marke\Downloads\labels (1).npy", "rb") as file:
    labels = np.load(file)

### Data Cleaning

In [4]:
data.shape

(87477, 9)

In [5]:
data.shape

(87477, 9)

In [6]:
data.shape
filtered = [[point[0],point[1]] for point in zip(data, labels) if np.sum(np.abs(point[0])) < 255*9]

In [7]:
len(filtered)

85132

In [8]:
data, label = zip(*filtered)

In [9]:
 X_train, X_test, y_train, y_test = train_test_split(data,label, test_size=0.2)

In [10]:
a = pd.Series(y_train).value_counts()
b = pd.Series(y_test).value_counts()

print(a)
print("\n")
print(b)

Not Ice    47494
Ice        20611
dtype: int64


Not Ice    11975
Ice         5052
dtype: int64


### Random Forest

In [11]:
fclf = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)

In [12]:
X_train

[array([1.83104143e-01, 1.79620236e-01, 1.81524277e-01, 1.63839132e-01,
        9.66395885e-02, 2.82468018e+02, 2.82171204e+02, 7.35853389e-02,
        1.88934848e-01]),
 array([2.17948779e-01, 2.15998843e-01, 2.29882300e-01, 2.16418415e-01,
        1.14015274e-01, 2.89160400e+02, 2.88989990e+02, 9.27910432e-02,
        2.32718721e-01]),
 array([5.68304300e-01, 6.25869751e-01, 6.05557799e-01, 6.08770132e-01,
        5.70115559e-02, 2.75942719e+02, 2.76249878e+02, 3.77907604e-02,
        6.70739174e-01]),
 array([1.20860189e-01, 1.11819491e-01, 1.07487872e-01, 9.79803726e-02,
        7.98750445e-02, 2.92888336e+02, 2.92833252e+02, 6.47686794e-02,
        1.06990173e-01]),
 array([1.36667877e-01, 1.08445831e-01, 1.00541823e-01, 9.46649387e-02,
        5.26702963e-02, 2.78959717e+02, 2.79252716e+02, 3.78881693e-02,
        1.24325365e-01]),
 array([5.34034491e-01, 5.48826933e-01, 5.66598773e-01, 5.30328751e-01,
        7.40093812e-02, 2.77160248e+02, 2.76924835e+02, 5.71101978e-02,
      

In [13]:
fclf.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, max_features=1, n_estimators=10)

In [14]:
fclf.score(X_test, y_test)

0.7450519762729783

In [15]:
fclf.predict(X_test)


array(['Not Ice', 'Not Ice', 'Not Ice', ..., 'Not Ice', 'Not Ice',
       'Not Ice'], dtype='<U7')

In [16]:
fclf.predict(X_test)

array(['Not Ice', 'Not Ice', 'Not Ice', ..., 'Not Ice', 'Not Ice',
       'Not Ice'], dtype='<U7')

In [17]:
sk.metrics.f1_score(y_test,fclf.predict(X_test), pos_label='Not Ice')

0.8330320396938343

### Decision Tree

In [33]:
dtr = lm.LogisticRegression(random_state=0).fit(X_train, y_train)

In [34]:
dtr.score(X_test, y_test)

0.6942503083338227

In [35]:
sk.metrics.f1_score(y_test,dtr.predict(X_test), pos_label='Not Ice')

0.8155862557562876

### SVM

In [189]:
sclf = SVC(gamma='auto')

In [ ]:
sclf.fit(X_train,y_train)

In [45]:
sclf.score(X_test, y_test)

0.7407261247040252

In [46]:
sk.metrics.f1_score(y_test,sclf.predict(X_test), pos_label='Not Ice')

0.8510541827250055

### Dimentionality Reduction

In [20]:
cclf = KMeans()

In [21]:
cclf.fit(X_train)

KMeans()

# Display prediction

In [42]:
full = 
ndsi =

NameError: name 'Xtrain' is not defined

In [51]:
# test_img = "Engilchek_glacier_2001-03-24.tif"
# test_img = r"Engilchek_glacier_1999-08-26.tif"
test_img = r"Engilchek_glacier_2018-08-30"

In [52]:
full_directory = os.path.join(r"C:\Users\marke\Documents\DSC180A-Q1\raw_data\full_img",test_img)
ndsi_directory = os.path.join(r"C:\Users\marke\Documents\DSC180A-Q1\raw_data\ndsi_imgs",test_img)

In [21]:
def runNapari(img_path, full_img_path):
    with rasterio.open(img_path) as src:
        img = src.read()
    
    with rasterio.open(full_img_path) as src:
        img_full = src.read()

    viewer = napari.view_image(img)
    return viewer, img, img_full


In [22]:
def loadAndTest(full_directory,ndsi_directory, trained_model):
    
    viewer,ndsi,full = runNapari(ndsi_directory,full_directory)
    e_dims = full.shape
    print(e_dims)
    testbals = np.rollaxis(full.reshape(9,e_dims[1]*e_dims[2]),0,2)
    print(testbals)
    prediction = trained_model.predict(testbals)

    prediction = prediction.reshape(e_dims[1],e_dims[2])
    # np.count_nonzero(testbals.reshape(9,145,363) != full)
    display_dims = prediction.shape
    img_coordinates = np.array(list(zip(np.arange(0,display_dims[0]),np.arange(0,display_dims[1]))))
    
    ice_indexes = np.where(prediction == "Ice")
    non_ice_indexes = np.where(prediction == "Not Ice")
    ice_coordinates = list(zip(ice_indexes[0],ice_indexes[1]))
    non_ice_coordinates = list(zip(non_ice_indexes[0], non_ice_indexes[1]))
    
    viewer = napari.view_image(ndsi)
    viewer.add_points(ice_coordinates,face_color="red",edge_color ="red",size=1, name="Ice")
    viewer.add_points(non_ice_coordinates,face_color="blue",edge_color ="blue",size=1, name="Not Ice")


In [53]:
loadAndTest(full_directory, ndsi_directory, fclf)

RasterioIOError: C:/Users/marke/Documents/DSC180A-Q1/raw_data/ndsi_imgs/Engilchek_glacier_2018-08-30: No such file or directory

### Scratch

In [271]:
sclf

SVC(gamma='auto')

In [169]:
full,ndsi = runNapari(test_img,full_directory,ndsi_directory)

In [247]:
e_dims = full.shape
testbals = np.rollaxis(full.reshape(9,e_dims[1]*e_dims[2]),0,2)
prediction = fclf.predict(testbals)

prediction = prediction.reshape(e_dims[1],e_dims[2])
# np.count_nonzero(testbals.reshape(9,145,363) != full)
display_dims = prediction.shape
display_dims

(145, 363)

In [239]:
img_coordinates = np.array(list(zip(np.arange(0,display_dims[1]),np.arange(0,display_dims[2]))))

In [250]:
ice_indexes = np.where(prediction == "Ice")
non_ice_indexes = np.where(prediction == "Not Ice")

In [251]:
ice_coordinates = list(zip(ice_indexes[0],ice_indexes[1]))
non_ice_coordinates = list(zip(non_ice_indexes[0], non_ice_indexes[1]))

In [253]:
viewer = napari.view_image(ndsi)

In [255]:
viewer.add_points(ice_coordinates,face_color="red",edge_color ="red",size=1, name="Ice")
viewer.add_points(non_ice_coordinates,face_color="blue",edge_color ="blue",size=1, name="Not Ice")


<Points layer 'Not Ice' at 0x21aaee09520>